<a href="https://colab.research.google.com/github/lis-r-barreto/Data-Engineering/blob/main/06_Building_a_PostgreSQL_database_for_crime_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a PostgreSQL database for crime reports

In this brief project we will cover the following aspects of managing databases with Postgres:

- How to create a database and manage database roles.
- How to create database schemas and tables with the proper datatypes.
- How to load data from CSV files into database tables.

We will put all this together to build a database for storing data related with crimes that occurred in Boston. This dataset is available in the file `boston.csv`.
Here we can see the first few rows of the file:

In [ ]:
import csv
import pandas as pd
import psycopg2

In [ ]:
data = pd.read_csv('boston.csv')
data.head()

,incident_number,offense_code,description,date,day_of_the_week,lat,long
0,1,619,LARCENY ALL OTHERS,2018-09-02,Sunday,42.357791,-71.139371
1,2,1402,VANDALISM,2018-08-21,Tuesday,42.306821,-71.060300
2,3,3410,TOWED MOTOR VEHICLE,2018-09-03,Monday,42.346589,-71.072429
3,4,3114,INVESTIGATE PROPERTY,2018-09-03,Monday,42.334182,-71.078664
4,5,3114,INVESTIGATE PROPERTY,2018-09-03,Monday,42.275365,-71.090361


As a description of the file:

- The first column represents the identifier of the crime. 
- The second contains a numeric identifier code for the committed crime. 
- The third represents a description of the crime. 
- The next two rows contain the date on which the crime happened and the corresponding day of the week. 
- The last two columns represent the location of the crime with a latitude and longitude coordinates.

## Road map

The goal of the project is to create a database named `crimes_db` with a table – `boston_crimes` – with appropriate datatypes for storing the data from the `boston.csv` file. 

We will be creating the table inside a schema named `crimes`. 
We will also create the `readonly` and `readwrite` groups with the appropriate privileges. 

Finally, we will also create one user for each of these groups.

## Create `crimes_db` database

In [ ]:
conn = psycopg2.connect(dbname='postgres', user='postgres')
cur = conn.cursor()
# set autocommit to True bacause this is required for creating databases
conn.autocommit = True
cur.execute("CREATE DATABASE crimes_db;")
conn.close()

In [ ]:
# Connect to the new database
conn = psycopg2.connect(dbname='crimes_db', user='postgres')
conn.autocommit = True
cur = conn.cursor()

In [ ]:
# Create the crimes schema
cur.execute("CREATE SCHEMA crimes;")
conn.commit()

## Obtaining the Column Names and Sample
Here we read the header row and assign it to a variable named `col_headers` and read the first data row and assign it to a variable named `first_row`.

In [ ]:
with open('boston.csv', 'r') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

In [ ]:
# Check the column names and sample
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Creating a function for analyzing column values¶

Before we create a table for storing the crime data, we need to identify the proper datatypes for the columns. To help us with that, let's create a function `get_col_set()` that given the name of a CSV file and a column index (starting at 0) computes a Python `Set` with all distinct values contained in that column.

This function will be useful for two reasons:

- Checking whether an `enumerated` datatype might be a good choice for representing a column.
- Computing the maximum length of any text-like column to select appropriate sizes for `VARCHAR` columns.

In [ ]:
# Create the function
def get_col_set(csv_filename, col_index):
    
    """Output: a Python set that contains all distinct values from the col_index in csv_filename."""
    
    distinct_values = set()
    with open(csv_filename) as file:
        reader = csv.reader(file)
        next(reader) # Skip headers row
        for row in reader:
            distinct_values.add(row[col_index])
    return distinct_values

- Let's compute the number of different values in each column of the boston.csv file.

In [ ]:
for col_index in range(7):
    number_distinct_values = len(get_col_set('boston.csv', col_index))
    print("{}: {}".format(col_headers[col_index], number_distinct_values))

incident_number: 298329
offense_code: 219
description: 239
date: 1177
day_of_the_week: 7
lat: 18177
long: 18177


## Analyzing the maximum length of the text columns¶

In order to pick the right datatypes for our table we need to know the maximum amount of characters in a single entry for each text column.

The `day_of_the_week` column contains only 7 values, one for each day, which makes it a good candidate for an enumerated datatype. 
We can tell that the longest of them is `Wednesday` without needing any computation.

For the `description` column we will reuse the function from the last step:

In [ ]:
# Check column index numbers
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [ ]:
# day_of_the_week column is at index 4
print(get_col_set('boston.csv', 4))

{'Tuesday', 'Monday', 'Friday', 'Sunday', 'Wednesday', 'Saturday', 'Thursday'}


In [ ]:
# description column is at index 2
description_column = get_col_set('boston.csv', 2)
max_value = 0
for value in description_column:
    max_value = max(len(value), max_value)
print('Maximum amount of characters for a single value:', max_value)

Maximum amount of characters for a single value: 58


## Creating the table


In [ ]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


We will use the same names for the column headers.

The amount of distinct values for each column was:

    incident_number 298329
    offense_code       219
    description        239
    date              1177
    day_of_the_week      7
    lat              18177
    long             18177
    
From the result of printing `first_row` we see that the different types of data that we have are:

    integer numbers
    integer numbers
    string
    date
    string
    decimal number
    decimal number
    
The `day_of_the_week` column has a small range of values so we will create an enumerated datatype named `WEEKDAY_ENUM`. The `offense_code` column would also be a good candidate since there is probably a limited set of possible offense codes.

For the `incident_number` column we have decided to use the type `INTEGER` and set it as the `PRIMARY KEY`. The same datatype will be used to represent the `offense_code` column.

We saw that the `description` column has size at most 59. To be on the safe side we will limit the size of the description to 100 and use the `VARCHAR(100)` datatype.

Finally, the `lat` and `long` columns seem to need to hold quite a lot of precision so we will use the `DECIMAL` type.

In [ ]:
# Create enumerated datatype for day_of_the_week
cur.execute("""CREATE TYPE weekday_enum AS ENUM (
            'Saturday', 'Thursday', 'Sunday', 'Wednesday', 'Monday', 'Friday', 'Tuesday');
""")

# Create table
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week WEEKDAY_ENUM,
        lat DECIMAL(11,8),
        long DECIMAL(11,8)
    );
""")

## Loading the data

In order to move the data from our file, `boston.csv`, to our newly created table we will use the `cursor.copy_expert` method.

There are many ways to load CSV data into a Postgres table. We could read the file and execute queries to add the lines one by one to our table. But the `cursor.copy_expert` method is more robust and much faster.

In [ ]:
# Load the data from boston.csv into the table boston_crimes in the crimes schema
with open('boston.csv') as file:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", file)

In [ ]:
cur.execute("SELECT * FROM crimes.boston_crimes")
# print the number of rows to ensure that they were loaded
print(len(cur.fetchall()))

298329


## Revoke public privileges¶

We will move on to creating our `readonly` and `readwrite` groups. Each group will have access to specific privileges according to their function.

By following the least privilege principle, the first step is to make sure that there are no privileges inherited from the `public group` and on the `public schema`.

In [ ]:
# Revoke privileges from public schema
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
# Revoke privileges from public group
cur.execute("REVOKE ALL ON DATABASE crimes_db FROM public;")

## Creating the `readonly` group¶

We create a `readonly` group with `NOLOGIN` because it is a group and not a user. We grant the group the ability to `CONNECT` to the `crimes_db` and the ability to use the `crimes` schema.

This group will only be able to use the `SELECT` command when quering.


In [ ]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

## Creating the `readwrite` group¶

We create a `readwrite` group with `NOLOGIN` because it is a group and not a user. We grant the group the ability to `CONNECT` to the `crimes_db` and the ability to use the `crimes` schema.

This group will have the SELECT, INSERT, UPDATE and DELETE priviliges.

In [ ]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

## Creating one user for each group¶

The last big step on this project will be to create one user in each group. We will need to create each user and then assign them to each group. 

- We create a user named `data_analyst` with password `secret1` in the `readonly` group.

- We create a user named `data_scientist` with password `secret2` in the `readwrite` group.

In [ ]:
# Create user and add it to readonly group
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")

# Create user and add it to readwrite group
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

## Test the database setup¶

It is a good practice to test that everything is configured as expected when we finish setting up the database.
To wrap up this project we will test the database setup using SQL queries to check whether the objects have been created and that users and groups have the right privileges.

This requires us to know the Postgres internal tables. We can query the `pg_roles` table to inspect privileges related to the database and the `information_schema.table_privileges` table to inspect table privileges.

In the `pg_roles` table we will check database related privileges. We will look at the following columns:

- `rolname`: The name of the user / group that the privilege refers to.
- `rolsuper`: Whether this user / group is a super user. It should be set to False on every user / group that we have created.
- `rolcreaterole`: Whether user / group can create users, groups or roles. It should be False on every user / group that we have created.
- `rolcreatedb`: Whether user / group can create databases. It should be False on every user / group that we have created.
- `rolcanlogin`: Whether user / group can login. It should be True on the users and False on the groups that we have created.

In [ ]:
# Close the old connection to test with a brand new connection
conn.close()

conn = psycopg2.connect(dbname="crimes_db", user="postgres")
cur = conn.cursor()
# Check users and groups
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
print('USERS AND GROUPS:')
for user in cur:
    print(user)
print()

USERS AND GROUPS:
('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)



In the `information_schema.table_privileges` we will check privileges related to SQL queries on tables. We will list the privileges of each group that we have created.

In [ ]:
# Check privileges
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
print('PRIVILEGES:')
for user in cur:
    print(user)
conn.close()

PRIVILEGES:
('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')


## Conclusion

In this project went through the basics of creating a database in PostgreSQL from a csv file.

As a summarization, we:
- Created our database
- Explored the data to shape our first table
- Created our table
- Loaded the data from the csv file to our database
- Created groups with customized privileges
- Created users inside these groups

This is of course just a way to kickstart our database which now has a lot of room for growing. More tables can be added and linked together. More users and group can be created. Much more data can be stored. 

And all these improvements will come with more challenges regarding keeping our database safe and organized. We will leave this for a future project.

## Thank you for reading!

